#Annotation Notebook
This notebook allows you to annotate your data with a number of annotation methods using the tabula sapiens dataset as a reference

Press shift+enter to execute each code block

Usage:
Here, we setup the Google Colab Environment, download the data, and connect to Google drive

Ad instructins on putting data on google drive etc.

# Setup 

In [1]:
#@title Import packages
#@markdown Here we install the necessary packages
import os
import warnings
import sys
warnings.simplefilter(action='ignore', category=FutureWarning)
import scanorama
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scvi
import scanpy as sc
import umap
from scipy import sparse
import scipy

sc.set_figure_params(figsize=(6, 6), frameon=False)
sc.settings.n_jobs=2


In [ ]:
# TODO
# Add code to download annotation code and datasets

In [4]:
#@title Connect to Google Drive
#@markdown Link this colab session to Google drive
from google.colab import drive
#mount google drive
drive.mount('/content/drive')

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: No module named 'google.colab'

In [5]:
#@title Import annotation code
import sys
sys.path.append('/content/drive/MyDrive/ts_evaluation') #TODO change this path
import importlib
import annotation 

importlib.reload(annotation)
from annotation import (process_query, 
                        save_results,
                        subsample_dataset,
                        run_scvi,
                        run_scanvi,
                        run_scanorama,
                        run_onclass,
                        run_knn_on_scvi,
                        run_knn_on_scanorama,
                        run_svm_on_hvg,
                        run_rf_on_hvg,
                        run_bbknn, 
                        run_knn_on_bbknn)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: No module named 'sklearn.externals.six'

## Load your data

Set the following arguments:

In [ ]:
# TODO set location to save results
save_folder = '/content/drive/MyDrive/Thymus_eval/notebook_evaluation'

# TODO set paths query dataset
query_path = '/content/drive/MyDrive/Lung_eval/Lung_lca_processed.h5ad'
# query_path = '/content/drive/MyDrive/Thymus_eval/thymus_query_processed.h5ad'
# query_path = '/content/drive/MyDrive/PBMC/pbmc_query.h5ad'

# TODO set batch key of input anndata
query_batch_key = 'method'


### setup for evaluation ###
ref_batch_key = 'method'
ref_path = '/content/drive/MyDrive/Lung_eval/Lung_ts_processed.h5ad'
# ref_path = '/content/drive/MyDrive/Thymus_eval/thymus_TS_processed.h5ad'
# ref_path = '/content/drive/MyDrive/PBMC/multi.new.h5ad'

#key to the label locations
#this is just for evaulation
ref_labels_key = 'cell_ontology_type'
query_labels_key = 'cell_ontology_type'

unknown_celltype_label = 'unknown'

#want to make this false in the future
training_method = 'offline'
results_adata_path = os.path.join(save_folder, 'results.h5ad')

cl_obo_file = '/content/drive/MyDrive/ts_evaluation/cl.obo'
cl_ontology_file = '/content/drive/MyDrive/ts_evaluation/cl.ontology'


!mkdir -p {save_folder}


In [ ]:
ref = anndata.read(ref_path)
query = anndata.read(query_path)

#delete later
query.obs[query_labels_key] = 'unknown'

# combine the query and reference datasets
adata = process_query(ref, 
                      query,
                      ref_labels_key,
                      ref_batch_key,
                      query_labels_key,
                      query_batch_key,
                      unknown_celltype_label = 'unknown',
                      training_method='offline')

All ref genes are in query dataset. Can use pretrained models
INFO     Using batches from adata.obs["_batch"]                                 
INFO     Using labels from adata.obs["_labels"]                                 
INFO     Using data from adata.layers["counts"]                                 
INFO     Computing library size prior per batch                                 
INFO     Successfully registered anndata object containing 107775 cells, 4000   
         vars, 4 batches, 24 labels, and 0 proteins. Also registered 0 extra    
         categorical covariates and 0 extra continuous covariates.              
INFO     Please do not further modify adata until model is trained.             


In [ ]:
adata

AnnData object with n_obs × n_vars = 107775 × 4000
    obs: 'cell_id', 'method', 'donor', 'cell_ontology_type', 'donor_method', 'cell_ontology_id', '_labels', '_batch', 'n_counts', 'n_genes', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variance

# Annotation methods

## bbknn

We run bbknn to perform batch correction, then run knn with the learned neighbors

In [ ]:
adata = run_bbknn(adata)
ref_idx = ref.obs_names
query_idx = query.obs_names
run_knn_on_bbknn(adata, 
                 train_idx=ref_idx,
                 test_idx=query_idx, 
                 labels_key='_labels',
                 result_key='knn_on_bbknn_pred')

Running bbknn
Classifying with knn on bbknn distances


/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:168: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  EfficiencyWarning)
/content/drive/MyDrive/ts_evaluation/annotation.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[result_key][test_idx] = knn_pred
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
adata

AnnData object with n_obs × n_vars = 268409 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'knn_on_bbknn_pred'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', 'GeneID-

In [ ]:
save_results(adata, results_adata_path, obs_keys=['knn_on_bbknn_pred'])

##scanorama
We run scanorama here 

In [ ]:
def cluster_and_subsample_cells(adata, use_rep, n_samples=100000):
  sc.pp.neighbors(adata, use_rep=use_rep)
  sc.tl.leiden(adata, key_added="leiden")
  train_idx = subsample_dataset(adata,'leiden', n_total_samples=n_samples)
  return train_idx

adata = anndata.read(results_adata_path)
adata = adata[query.obs_names].copy()
train_idx = cluster_and_subsample_cells(adata, use_rep='X_scvi')
ref_and_subsetted_query = np.concatenate((ref.obs_names.values,train_idx.values))
adata = adata[ref_and_subsetted_query].copy()

['0' '1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21'
 '22' '23' '24' '25' '26' '27' '28' '29' '3' '30' '31' '32' '33' '34' '35'
 '36' '37' '38' '39' '4' '40' '41' '42' '5' '6' '7' '8' '9']
Sampling 2325 per label


In [ ]:
ref = anndata.read(ref_path)
query = anndata.read(query_path)

#delete later
query.obs[query_labels_key] = 'unknown'

# combine the query and reference datasets
adata = process_query(ref, 
                      query,
                      ref_labels_key,
                      ref_batch_key,
                      query_labels_key,
                      query_batch_key,
                      unknown_celltype_label = 'unknown',
                      training_method='offline')

All ref genes are in query dataset. Can use pretrained models
INFO     Using batches from adata.obs["_batch"]                                 
INFO     Using labels from adata.obs["_labels"]                                 
INFO     Using data from adata.layers["counts"]                                 
INFO     Computing library size prior per batch                                 
INFO     Successfully registered anndata object containing 268409 cells, 4000   
         vars, 4 batches, 24 labels, and 0 proteins. Also registered 0 extra    
         categorical covariates and 0 extra continuous covariates.              
INFO     Please do not further modify adata until model is trained.             


In [ ]:
print(ref)

AnnData object with n_obs × n_vars = 12582 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'GeneName-1', 'GeneID-1', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'hvg'


In [ ]:
print(adata)

AnnData object with n_obs × n_vars = 268409 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', 'GeneID-1

In [ ]:
ref_and_subsetted_query = np.concatenate((ref.obs_names.values,subsetted_query_idx.values))


In [ ]:
ref_and_subsetted_query

array(['AAACCCACACCTGAAT_TSP2_Thymus_NA_10X_1_1-12-0',
       'AAACCCAGTTTAGACC_TSP2_Thymus_NA_10X_1_1-12-0',
       'AAACGAAAGGATACCG_TSP2_Thymus_NA_10X_1_1-12-0', ...,
       '5478STDY7935100-AGGTCCGCAATGTTGC-1',
       'WSSS8062670-CTCTGGTTCCAGTAGT-1',
       'T06_TH_TOT_5GEX_3_S11-CAGCTGGTCCCGGATG-1'], dtype=object)

In [ ]:
adata[subsetted_adata.obs_names]

View of AnnData object with n_obs × n_vars = 112582 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', '

In [ ]:
adata[ref_and_subsetted_query]

View of AnnData object with n_obs × n_vars = 112582 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', '

In [ ]:
subsetted_adata = adata[ref_and_subsetted_query].copy()


In [ ]:
subsetted_adata[subsetted_query_idx]

View of AnnData object with n_obs × n_vars = 100000 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', '

In [ ]:
subsetted_adata = run_scanorama(subsetted_adata)


Found 4000 genes among all datasets
[[0.         0.25049718 0.19655287 0.87159533]
 [0.         0.         0.64584906 0.14007782]
 [0.         0.         0.         0.35992218]
 [0.         0.         0.         0.        ]]
Processing datasets (0, 3)
Processing datasets (1, 2)
Processing datasets (2, 3)
Processing datasets (0, 1)
Processing datasets (0, 2)
Processing datasets (1, 3)


In [ ]:
subsetted_adata[subsetted_query_idx]


View of AnnData object with n_obs × n_vars = 12582 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', 'G

In [ ]:
train_idx = ref.obs_names
test_idx = subsetted_query_idx
run_knn_on_scanorama(subsetted_adata, train_idx, test_idx)

/content/drive/MyDrive/ts_evaluation/annotation.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[result_key][test_idx] = knn_pred
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
subsetted_adata

AnnData object with n_obs × n_vars = 112582 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'knn_on_scanorama_pred'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0'

In [ ]:
adata.obs['knn_on_scanorama_pred'] = 'na'

In [ ]:
adata.obs['knn_on_scanorama_pred'][subsetted_adata.obs_names] = subsetted_adata.obs['knn_on_scanorama_pred']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
from scipy import sparse

adata.obsm['X_scanorama'] = sparse.csr_matrix((adata.n_obs, 50))

In [ ]:
import pandas as pd
df = pd.DataFrame(index = adata.obs_names, data = np.zeros(shape=(adata.n_obs, 50)))
df.loc[subsetted_adata.obs_names]=subsetted_adata.obsm['X_scanorama']

In [ ]:
adata[subsetted_adata.obs_names].obsm['X_scanorama'] = subsetted_adata.obsm['X_scanorama']

In [ ]:
adata.obsm['X_scanorama']= df

In [ ]:
x_scanorama = sparse.csr_matrix(df.values)
adata.obsm['X_scanorama']=x_scanorama

In [ ]:
adata.obsm['X_scanorama']

<268409x50 sparse matrix of type '<class 'numpy.float64'>'
	with 5629100 stored elements in Compressed Sparse Row format>

In [ ]:
adata = adata[subsetted_adata.obs_names].copy()

In [ ]:
tmp_path = results_adata_path[:-12] + "subsetted_results.h5ad"

In [ ]:
res = anndata.read(results_adata_path)

In [ ]:
res = res[subsetted_adata.obs_names].copy()

In [ ]:
res.write(tmp_path, 'gzip')

In [ ]:
tmp_path

'/content/drive/MyDrive/Thymus_eval/notebook_evaluation/subsetted_results.h5ad'

In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['knn_on_scanorama_pred'], 
             obsm_keys=['X_scanorama'])

... storing 'knn_on_scanorama_pred' as categorical


In [ ]:
adata

AnnData object with n_obs × n_vars = 268409 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'knn_on_scvi_pred'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'Ge

In [ ]:
res.obs_names[0] in subsetted_adata.obs_names

True

In [ ]:
# adata_subset = adata.copy()
# adata_subset.obsm['X_scanorama']

# import pandas as pd
# df = pd.DataFrame(index = adata.obs_names, data = np.zeros(shape=(adata.n_obs, 50)))
# df.loc[adata_subset.obs_names]=adata_subset.obsm['X_scanorama']
# from scipy import sparse
# x_scanorama = sparse.csr_matrix(df.values)
# adata.obsm['X_scanorama']=x_scanorama

# adata.obs['knn_on_scanorama_pred'] = adata_subset.obs['knn_on_scanorama_pred']
# adata.obs['knn_on_scanorama_pred']
# save_results(adata, 
#              results_adata_path, 
#              obs_keys=['knn_on_scanorama_pred'], 
#              obsm_keys=['X_scanorama'])

In [ ]:
#TODO check and run subsampling
adata = run_scanorama(adata)

Found 4000 genes among all datasets
[[0.         0.24361949 0.20931389 0.87548638]
 [0.         0.         0.64352374 0.11867704]
 [0.         0.         0.         0.3540856 ]
 [0.         0.         0.         0.        ]]
Processing datasets (0, 3)
Processing datasets (1, 2)
Processing datasets (2, 3)
Processing datasets (0, 1)
Processing datasets (0, 2)
Processing datasets (1, 3)


In [ ]:
adata

AnnData object with n_obs × n_vars = 112582 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', 'GeneID-1

In [ ]:
train_idx = ref.obs_names
test_idx = subsetted_query_idx.values
run_knn_on_scanorama(adata, train_idx, test_idx)

KeyError: ignored

In [ ]:
adata

AnnData object with n_obs × n_vars = 112582 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', 'GeneID-1

In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['knn_on_scanorama_pred'], 
             obsm_keys=['X_scanorama'])

In [ ]:
# adata_subset = adata.copy()
# adata_subset.obsm['X_scanorama']

# import pandas as pd
# df = pd.DataFrame(index = adata.obs_names, data = np.zeros(shape=(adata.n_obs, 50)))
# df.loc[adata_subset.obs_names]=adata_subset.obsm['X_scanorama']
# from scipy import sparse
# x_scanorama = sparse.csr_matrix(df.values)
# adata.obsm['X_scanorama']=x_scanorama

# adata.obs['knn_on_scanorama_pred'] = adata_subset.obs['knn_on_scanorama_pred']
# adata.obs['knn_on_scanorama_pred']
# save_results(adata, 
#              results_adata_path, 
#              obs_keys=['knn_on_scanorama_pred'], 
#              obsm_keys=['X_scanorama'])

## scvi offline

In [ ]:
importlib.reload(annotation)
from annotation import (process_query,run_scvi)
run_scvi(adata, n_latent=50, dropout_rate=0.1, dispersion='gene-batch')

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/30:   0%|          | 0/30 [00:03<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id

In [ ]:
train_idx = ref.obs_names
test_idx = query.obs_names
run_knn_on_scvi(adata,
                train_idx,
                test_idx, 
                obsm_key='X_scvi',
                result_key='knn_on_scvi_pred')

/content/drive/MyDrive/ts_evaluation/annotation.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[result_key][test_idx] = knn_pred
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
sc.pp.neighbors(adata, use_rep="X_scvi")
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(
    adata,
    color=["method", "cell_ontology_type"],
    ncols=1
)

In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['knn_on_scvi_pred'], 
             obsm_keys=['X_scvi'])


## scvi online

In [ ]:
importlib.reload(annotation)
from annotation import run_scvi
scvi.data.setup_anndata(ref, batch_key='_batch', labels_key='_labels')

INFO     Using batches from adata.obs["_batch"]                                 
INFO     Using labels from adata.obs["_labels"]                                 
INFO     Using data from adata.X                                                
INFO     Computing library size prior per batch                                 
INFO     Successfully registered anndata object containing 32704 cells, 4000    
         vars, 2 batches, 23 labels, and 0 proteins. Also registered 0 extra    
         categorical covariates and 0 extra continuous covariates.              
INFO     Please do not further modify adata until model is trained.             


In [ ]:
run_scvi(ref, n_latent=50, dropout_rate=0.1, dispersion='gene-batch', 
         save_folder='/content/drive/MyDrive/Lung_eval/scvi_model')

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


suh
Epoch 1/245:   0%|          | 0/245 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


Epoch 245/245: 100%|██████████| 245/245 [03:29<00:00,  1.17it/s, loss=1.01e+03, v_num=1]


In [ ]:
def train_online_scvi(adata, 
                      pretrained_scvi_path,
                      max_epochs=None,
                      save_path=None,
                      overwrite=True):
  if max_epochs is None:
    n_cells = adata.n_obs
    max_epochs = np.min([round((20000 / n_cells) * 400), 400])
    max_epochs = int(max_epochs/2)
  
  model = scvi.model.SCVI.load_query_data(adata,
                                          pretrained_scvi_path)
  model.train(max_epochs=max_epochs, 
              train_size=1.0)
  adata.obsm['X_scvi'] = model.get_latent_representation()
  if save_path is not None:
    model.save(save_path, overwrite=overwrite)


In [ ]:
train_online_scvi(query,
                  '/content/drive/MyDrive/Lung_eval/scvi_model', 
                  save_path='/content/drive/MyDrive/Lung_eval/scarches_model')

INFO     Using data from adata.X                                                
INFO     Computing library size prior per batch                                 
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var',  
         'labels']                                                              
INFO     Successfully registered anndata object containing 75071 cells, 4000    
         vars, 4 batches, 24 labels, and 0 proteins. Also registered 0 extra    
         categorical covariates and 0 extra continuous covariates.              


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/53:   0%|          | 0/53 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


Epoch 53/53: 100%|██████████| 53/53 [07:12<00:00,  8.15s/it, loss=1.21e+03, v_num=1]


In [ ]:
query

AnnData object with n_obs × n_vars = 75071 × 4000
    obs: 'cell_id', 'method', 'donor', 'cell_ontology_type', 'donor_method', 'cell_ontology_id', 'n_counts', 'n_genes', '_batch', '_labels', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0'
    uns: '_scvi'
    obsm: 'X_scvi'

In [ ]:
def run_knn_on_scvi(adata,
                    train_idx,
                    test_idx, 
                    obsm_key='X_scvi',
                    result_key='knn_on_scvi_pred'):
  
  if obsm_key not in adata.obsm.keys(): 
    print('Please train scVI first or pass in a valid obsm_key.')
  
  train_X = adata[train_idx].obsm[obsm_key]
  test_X = adata[test_idx].obsm[obsm_key]

  train_Y = adata[train_idx].obs['_labels'].to_numpy()
  knn = KNeighborsClassifier(n_neighbors = 15, weights='uniform')
  knn.fit(train_X, train_Y)
  knn_pred= knn.predict(test_X)

  #save_results
  adata.obs[result_key] = 'na'
  adata.obs[result_key][test_idx] = knn_pred

In [ ]:
obsm_key = 'X_scvi'
train_X = ref.obsm[obsm_key]
train_Y = ref.obs['_labels'].to_numpy()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 15, weights='uniform')


In [ ]:
knn.fit(train_X, train_Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [ ]:
test_X = query.obsm[obsm_key]
knn_pred= knn.predict(test_X)

In [ ]:
query.obs['knn_on_scvi_scarches_pred'] = knn_pred

In [ ]:
query.write('/content/drive/MyDrive/Lung_eval/scarches_results.h5ad', 'gzip')

... storing 'cell_ontology_type' as categorical
... storing '_batch' as categorical
... storing '_labels' as categorical
... storing 'knn_on_scvi_scarches_pred' as categorical


## scanvi offline

In [ ]:
adata

AnnData object with n_obs × n_vars = 33135 × 3463
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', 'cell_ontology_id', '_labels', '_batch', 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    var: 'feature_types.0.0-0-0', 'n_cells.0.0-0-0', 'gene_symbol.0.0-0-0', 'n_cells.1.0-0-0', 'n_cells.0-0-0', 'n_cells.1.1-0-0', 'feature_types.0.0.0.1-0-0', 'gene_symbol.0.0.0.1-0-0', 'n_cells.1.0.0.1-0-0', 'n_cells.1.0.1-0-0', 'n_cells-0-0', 'len-0-0', 'ensembl_id-0-0', 'contamination_prop-0-0-0', 'contamination_prop-1-0-0', 'contamination_prop-10-0-0', 'contamination_prop-11-0-0', 'contamination_prop-12-0-0', 'contamination_prop-13-0-0', 'contamination_prop-14-0-0', 'contamination_prop-2-0-0', 'contamination_prop-3-0-0', 'contamination_prop-4-0-0', 'contamination_prop-5-0-0', 'contamination_prop-6-0-0', '

In [ ]:
run_scanvi(adata, n_latent=100, dropout_rate=0.1, 
           n_epochs_semisupervised=1,
           n_epochs_unsupervised=1, 
           save_folder = 'thymus_scanvi')

INFO     Training Unsupervised Trainer for 1 epochs.                            
INFO     Training SemiSupervised Trainer for 1 epochs.                          


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/1:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


Epoch 1/1: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s, loss=1.18e+03, v_num=1]

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Epoch 1/1: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it, loss=1.25e+03, v_num=1]


In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['scanvi_pred'])

## scanvi online

In [ ]:
scvi.data.setup_anndata(ref, batch_key='_batch', labels_key='_labels')

INFO     Using batches from adata.obs["_batch"]                                 
INFO     Using labels from adata.obs["_labels"]                                 
INFO     Using data from adata.X                                                
INFO     Computing library size prior per batch                                 
INFO     Successfully registered anndata object containing 12582 cells, 4000    
         vars, 2 batches, 23 labels, and 0 proteins. Also registered 0 extra    
         categorical covariates and 0 extra continuous covariates.              
INFO     Please do not further modify adata until model is trained.             


In [ ]:
run_scanvi(ref, n_latent=100, dropout_rate=0.1, 
           n_epochs_semisupervised=None,
           n_epochs_unsupervised=None, 
           save_folder = 'thymus_scanvi')


INFO     Training Unsupervised Trainer for 400 epochs.                          
INFO     Training SemiSupervised Trainer for 400 epochs.                        


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/400:   0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


Epoch 400/400: 100%|██████████| 400/400 [03:33<00:00,  1.88it/s, loss=964, v_num=1]

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Epoch 400/400: 100%|██████████| 400/400 [06:06<00:00,  1.09it/s, loss=1.06e+03, v_num=1]


In [ ]:
def train_online_scanvi(adata, 
                        pretrained_scanvi_path,
                        max_epochs=None,
                        trainer_kwargs=None,
                        n_samples_per_label=100,
                        save_path=None,
                        overwrite=True):
  if max_epochs is None:
    n_cells = adata.n_obs
    max_epochs = np.min([round((20000 / n_cells) * 400), 400])
    max_epochs = int(max_epochs/2)

  model = scvi.model.SCANVI.load_query_data(adata, 
                                            pretrained_scanvi_path, 
                                            freeze_classifier=True)
  model.train(max_epochs = max_epochs, 
              train_size =1.0,
              n_samples_per_label=n_samples_per_label)
  if save_path is not None:
    model.save(save_path, overwrite=overwrite)

In [ ]:
train_online_scanvi(query,
                    pretrained_scanvi_path='thymus_scanvi/scanvi_model', 
                    semisupervised_trainer_kwargs=dict(weight_decay=0.0),
                    save_path='/content/drive/MyDrive/Thymus_eval/scanvi_online')

INFO     Using data from adata.X                                                
INFO     Computing library size prior per batch                                 
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var',  
         'labels']                                                              
INFO     Successfully registered anndata object containing 255827 cells, 4000   
         vars, 4 batches, 24 labels, and 0 proteins. Also registered 0 extra    
         categorical covariates and 0 extra continuous covariates.              
INFO     Training Unsupervised Trainer for 31 epochs.                           
INFO     Training SemiSupervised Trainer for 10 epochs.                         


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/10:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


Epoch 10/10: 100%|██████████| 10/10 [02:14<00:00, 13.46s/it, loss=1.8e+03, v_num=1]


In [ ]:
m = scvi.model.SCANVI.load('/content/drive/MyDrive/Thymus_eval/scanvi_online/', query)
adata.obs['scanvi_online_pred']= 'na'
adata.obs['scanvi_online_pred'][query.obs_names] = m.predict()

INFO     Using data from adata.X                                                
INFO     Computing library size prior per batch                                 
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var',  
         'labels']                                                              
INFO     Successfully registered anndata object containing 255827 cells, 4000   
         vars, 4 batches, 24 labels, and 0 proteins. Also registered 0 extra    
         categorical covariates and 0 extra continuous covariates.              


In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['scanvi_online_pred'])

... storing 'scanvi_online_pred' as categorical


## SVM on HVGs

In [ ]:
train_idx = subsample_dataset(adata[ref.obs_names],ref_labels_key)
run_svm_on_hvg(adata, train_idx, query.obs_names)

(1827, 4000)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/content/drive/MyDrive/ts_evaluation/annotation.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['svm_pred'][test_idx] = svm_pred
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


AnnData object with n_obs × n_vars = 268409 × 4000
    obs: 'donor', 'method', 'donor_method', 'cell_ontology_type', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'svm_pred'
    var: 'feature_types.0.0-0', 'n_cells.0.0-0', 'gene_symbol.0.0-0', 'n_cells.1.0-0', 'n_cells.0-0', 'n_cells.1.1-0', 'feature_types.0.0.0.1-0', 'gene_symbol.0.0.0.1-0', 'n_cells.1.0.0.1-0', 'n_cells.1.0.1-0', 'n_cells-0', 'len-0', 'ensembl_id-0', 'contamination_prop-0-0', 'contamination_prop-1-0', 'contamination_prop-10-0', 'contamination_prop-11-0', 'contamination_prop-12-0', 'contamination_prop-13-0', 'contamination_prop-14-0', 'contamination_prop-2-0', 'contamination_prop-3-0', 'contamination_prop-4-0', 'contamination_prop-5-0', 'contamination_prop-6-0', 'contamination_prop-7-0', 'contamination_prop-8-0', 'contamination_prop-9-0', 'highly_variable-0', 'highly_variable_rank-0', 'means-0', 'variances-0', 'variances_norm-0', 'GeneName-1', 'GeneID-1', 'mean

In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['svm_pred'])

... storing 'svm_pred' as categorical


## Random Forest on HVGs

In [ ]:
train_idx = subsample_dataset(adata[ref.obs_names],ref_labels_key)
run_rf_on_hvg(adata, train_idx, query.obs_names, save_key='rf_on_hvg_pred')

Training random forest classifier with ['CD8-positive, alpha-beta T cell' 'CD8-positive, alpha-beta T cell'
 'CD8-positive, alpha-beta T cell' ... 'thymocyte' 'thymocyte' 'thymocyte'] cells


/content/drive/MyDrive/ts_evaluation/annotation.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[save_key][test_idx] = rf_pred
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['rf_on_hvg_pred'])

... storing 'rf_on_hvg_pred' as categorical


## OnClass

In [ ]:
print(ref_path)
print(query_path)

run_onclass(adata=adata, 
            max_iter=2,
            ref_adata_path=ref_path,
            query_adata_path=query_path, 
            cl_obo_file=cl_obo_file, 
            cl_ontology_file=cl_ontology_file)

/content/drive/MyDrive/PBMC/multi.new.h5ad
/content/drive/MyDrive/PBMC/pbmc_query.h5ad
init OnClass
100.000000 precentage of labels are in the Cell Ontology


/usr/local/lib/python3.6/dist-packages/numpy/lib/format.py:350: UserWarning: metadata on a dtype may be saved or ignored, but will raise if saved when read. Use another form of storage.
  d['descr'] = dtype_to_descr(array.dtype)


Cost after epoch 1: loss:3180.003 acc: 0.875 auc: 0.996
Cost after epoch 2: loss:1536.538 acc: 0.883 auc: 0.996
training finished
no label file is provided
5527
number of intersection genes 20713
5527 5527


/content/drive/MyDrive/ts_evaluation/annotation.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs['onclass_pred'][test_adata.obs_names]=test_adata.obs['onclass_pred']
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


AnnData object with n_obs × n_vars = 26080 × 4000
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'cell_ontology_type', 'method', 'cell_ontology_id', '_labels', '_batch', 'batch', '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var', 'onclass_pred'
    var: 'features-0', 'gene_ids-1', 'feature_types-1', 'genome-1', 'mean', 'std', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'log1p', 'hvg', '_scvi'
    layers: 'counts'

In [ ]:
save_results(adata, 
             results_adata_path, 
             obs_keys=['onclass_pred'])

... storing 'onclass_pred' as categorical


# Statistics

In [ ]:
# save results to this directory
res_path = '/content/drive/MyDrive/Thymus_eval/output/'
sc.settings.figdir = res_path

## Compute Agreement and Disagreement

In [ ]:
# TODO change to path of results
# results = anndata.read('/content/drive/MyDrive/Lung_eval/new_api_ts_as_ref/results.h5ad')
results = anndata.read(results_adata_path)

In [ ]:
results.obs['cell_ontology_type'] = adata.obs['cell_ontology_type']

In [ ]:
methods = ['knn_on_bbknn_pred',
 'knn_on_scanorama_pred',
 'knn_on_scvi_pred',
 'scanvi_pred',
 'onclass_pred',
 'svm_on_hvg_pred',
 'rf_on_hvg_pred']

labels = results.obs['cell_ontology_type'].str.lower()

### make all celltype names lower case to accommadate onclass predictions

In [ ]:
for x in methods:
    results.obs[x] = [y.lower() for y in results.obs[x]]

In [ ]:
ref_cells = results.obs['_labels'].values!='unknown'

### Overall agreement


In [ ]:
agreement = results.obs[methods].apply(MajorityCount, axis=1)
agreement[ref_cells] = 0 
results.obs["majority_pred"] = results.obs[methods].apply(MajorityVote, axis=1)
results.obs["agreement"] = agreement.values.round(2).astype(str)

In [ ]:
#@markdown this computes the percentage of cells that the algorithms agree/disagree on 
agreement_counts = pd.DataFrame(
    np.unique(results[~ref_cells].obs["agreement"], return_counts=True)
).T

agreement_counts[0] = np.round(agreement_counts[0].astype(float) * 7).astype(int)
agreement_counts["freq"] = agreement_counts[1] / np.sum(agreement_counts[1])
agreement_counts.columns = ["n_agree", "count", "freq"]
display(agreement_counts)

,n_agree,count,freq
0,1,1,1.33207e-05
1,2,57,0.000759281
2,3,586,0.00780594
3,4,1849,0.02463
4,5,3155,0.0420269
5,6,13097,0.174462
6,7,56326,0.750303


### which algorithm tends to be the disagreement? 

In [ ]:
agreement_counts.to_csv(res_path + 'agreement_counts.csv')

FileNotFoundError: ignored

### per celltype agreement

In [ ]:
#@ This computes what ground truth celltypes the algorithm disagrees on the most
disagreement = {}
for x in np.unique(labels[~ref_cells]):
  temp = pd.DataFrame(np.unique(results[(~ref_cells) & (labels == x)].obs["agreement"],return_counts=True))
  temp.columns = temp.loc[0]
  temp = temp.loc[1]
  disagreement[x] = temp

disagreement = pd.DataFrame.from_dict(disagreement).T
disagreement.fillna(0, inplace=True)
disagreement = (disagreement.T / disagreement.sum(axis=1)).T.sort_values('1.0')
disagreement.style.format('{0:,.2f}').background_gradient(axis=0) 
disagreement.to_csv(res_path + "per_celltype_agreement.csv")

KeyError: ignored

In [ ]:
disagreement

""
NaN


In [ ]:
### import cell ontology 


### import cell ontology 


In [ ]:
import obonet

obo = "/content/drive/MyDrive/ts_evaluation/" + "cl.obo"
f = open(obo, "r")
co = obonet.read_obo(f)
f.close()

celltype_dict = {}
for x in co.nodes:
    celltype_dict[co.nodes[x]["name"].lower()] = x


In [ ]:
results.obsm['X_umap'] = results.obsm['X_scvi_50latent_0.1dropout_genebatchdispersion_umap']
CM = {}
ACC1 = {}
ACC2 = {}
MEAN = {}
for name in methods + ['majority_pred']:
    pred = results.obs[name].values

    df, prop = PredictionEval(
      pred[~ref_cells], labels[~ref_cells], res_path, name=name
    )

    acc1, acc2, acc_detail, mean1, mean2 = EvalSummary(
        df, prop.loc[df.index, "prop"].values, co
    )

    print(name, "weighted mean accuracy:", mean2, "unweighted mean accuracy", np.mean(acc2))

    acc_bar = accuracy_barplot(
        df, acc1, acc2, prop.loc[df.index, "prop"].values, name, res_path
    )
    ACC1[name] = acc1
    ACC2[name] = acc2
    MEAN[name] = [mean1, mean2, np.mean(acc1), np.mean(acc2)]
    CM[name] = df
    sc.pl.umap(
        results[~ref_cells],
        color=[name],
        save = 'scVI_pred_%s.pdf'%name,
        title = 'UMAP scVI Prediction %s'%name
    )


## Consensus Statistics

In [ ]:
labels = results.obs["cell_ontology_type"].astype(str).str.lower()

In [ ]:
# TODO add consensus statistics code

## Summary of cell type compositions

In [ ]:
# TODO add cell type composition code here